In [1]:
# !wget https://storage.googleapis.com/tfhub-modules/google/albert_base/2.tar.gz
# !tar -zxf 2.tar.gz

In [2]:
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/text-classification/data.zip
# !unzip data.zip

In [3]:
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/text-classification/utils.py

In [4]:
# !pip3 install albert-tensorflow

In [5]:
!ls assets

30k-clean.model  30k-clean.vocab  albert_config.json


In [6]:
!ls variables

variables.data-00000-of-00001  variables.index


In [7]:
from albert import modeling
from albert import optimization
from albert import tokenization
import tensorflow as tf
import numpy as np

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [8]:
tokenizer = tokenization.FullTokenizer(
      vocab_file='assets/30k-clean.vocab', do_lower_case=True,
      spm_model_file='assets/30k-clean.model')


INFO:tensorflow:loading sentence piece model


In [9]:
tokenizer.tokenize('husein is so cute')

['▁hus', 'ein', '▁is', '▁so', '▁cute']

In [10]:
from utils import *
from sklearn.model_selection import train_test_split

In [11]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [12]:
MAX_SEQ_LENGTH = 100

In [13]:
from tqdm import tqdm

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(trainset.data):
    tokens_a = tokenizer.tokenize(text.lower())
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_id = [0] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    padding = [0] * (MAX_SEQ_LENGTH - len(input_id))
    input_id += padding
    input_mask += padding
    segment_id += padding
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 10662/10662 [00:00<00:00, 11143.98it/s]


In [14]:
albert_config = modeling.AlbertConfig.from_json_file('assets/albert_config.json')
albert_config

In [15]:
tokenizer.tokenize(trainset.data[0])

['▁moving', '▁uneven', '▁success']

In [16]:
epoch = 10
batch_size = 32
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [17]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.AlbertModel(
            config=albert_config,
            is_training=False,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
dimension_output = 2
learning_rate = 5e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())





Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:++++++ warmup starts at step 0, for 333 steps ++++++
INFO:tensorflow:using adamw

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
var_lists

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(30000, 128) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 128) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 128) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/embedding_hidden_mapping_in/kernel:0' shape=(128, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/embedding_hidden_mapping_in/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/kernel:0' shape=(768, 7

In [20]:
saver.restore(sess, 'variables/variables')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from variables/variables


In [21]:
train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, trainset.target, test_size = 0.2
)

In [22]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_input_ids) / batch_size
    train_acc /= len(train_input_ids) / batch_size
    test_loss /= len(test_input_ids) / batch_size
    test_acc /= len(test_input_ids) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.737615
time taken: 65.3192994594574
epoch: 0, training loss: 0.588583, training acc: 0.693689, valid loss: 0.545611, valid acc: 0.737615



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.737615, current acc: 0.777711
time taken: 61.85024404525757
epoch: 1, training loss: 0.557109, training acc: 0.742843, valid loss: 0.498011, valid acc: 0.777711



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

...

time taken: 61.82991981506348
epoch: 6, training loss: 0.200658, training acc: 0.948308, valid loss: 0.842970, valid acc: 0.795057



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

time taken: 61.805060386657715
epoch: 7, training loss: 0.169402, training acc: 0.958860, valid loss: 0.827865, valid acc: 0.791307



test minibatch loop: 100%|██████████| 67/67 [00:05<00:00, 13.27it/s, accuracy=0.762, cost=1.04] 

time taken: 61.79657506942749
epoch: 8, training loss: 0.127402, training acc: 0.971992, valid loss: 0.854281, valid acc: 0.790615

break epoch:9

